In [1]:
from sklearn import model_selection
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
import os
import numpy as np
import pandas as pd
from MicroBiome import MicroBiomeDataSet, Trainer, TrainTester, MultiTrainTester
from SNN import FeedForward, SiameseDataSet, SiameseModel
import seaborn as sns

# Ignore warning messages
if True:
    import warnings
    warnings.filterwarnings('ignore')

# Load Data

In [2]:
iris = datasets.load_iris()
X = iris.data 
y = iris.target
OneHot = OneHotEncoder(sparse=False)
y = OneHot.fit_transform(y.reshape((y.shape[0], 1)))

In [3]:
y

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

# Define and Train Model

In [4]:
MyFeedForward = FeedForward(3, [3, 3, 3])

In [5]:
MyFeedForward

FeedForward(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=3, bias=True)
    (1): Linear(in_features=3, out_features=3, bias=True)
    (2): Linear(in_features=3, out_features=3, bias=True)
  )
)

In [6]:
MySNN = SiameseModel(MyFeedForward, predict_unknown = False)

In [7]:
# list(MySNN.logistic.parameters()) + list(MySNN.model.parameters())

In [8]:
MyTrainer = Trainer(model = MySNN, scale_X = True, use_pca = True, n_components = 3)

In [9]:
MyTrainTester = TrainTester(MyTrainer, metrics.accuracy_score)

In [10]:
MyTrainTester.train(X, y)

#########################################
Epoch 1 of 5
__Training__
2021-03-05 22:22:26
Batch Mean Loss: 0.671798050403595
Batch Mean Loss: 0.6702224016189575
Batch Mean Loss: 0.6738988757133484
Batch Mean Loss: 0.6698285341262817
Batch Mean Loss: 0.6720606088638306
Batch Mean Loss: 0.6766561269760132
Batch Mean Loss: 0.6752117872238159
Batch Mean Loss: 0.6733735799789429
Batch Mean Loss: 0.6741613149642944
Batch Mean Loss: 0.6742926836013794
Batch Mean Loss: 0.6758682727813721
Batch Mean Loss: 0.6742927432060242
Batch Mean Loss: 0.6769186854362488
Batch Mean Loss: 0.676393449306488
Batch Mean Loss: 0.6717979907989502
Batch Mean Loss: 0.6689093708992004
Batch Mean Loss: 0.6754744052886963
Batch Mean Loss: 0.671798050403595
Batch Mean Loss: 0.6736361980438232
Batch Mean Loss: 0.6748179197311401
__Validation__
2021-03-05 22:22:26
Batch Mean Loss: 0.6741613745689392
Batch Mean Loss: 0.6744239330291748
Batch Mean Loss: 0.6692773103713989
#########################################
Epoch 2 of

In [12]:
y_pred_train = MyTrainTester.Trainer.predict(MyTrainTester.X_train)

In [13]:
y_pred_train

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1,

In [ ]:
y_pred_train_1D = 